###### Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license (c) 2019 Daniel Koehn, based on (c)2018 L.A. Barba, G.F. Forsyth [CFD Python](https://github.com/barbagroup/CFDPython#cfd-python), (c)2014 L.A. Barba, I. Hawke, B. Knaepen [Practical Numerical Methods with Python](https://github.com/numerical-mooc/numerical-mooc#practical-numerical-methods-with-python), also under CC-BY.

In [1]:
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, 'r').read())

# 1D Heat Conduction Example I: Diurnal periodic heating of a half-space

In the last notebook we dicussed different types of boundary conditions, like the Dirichlet condition (fixed values at the boundaries) or the Neumann condition (fixed gradient values at the boundaries). In this exercise we will apply a diurnal periodic heating as boundary condition to compute the temperature distribution in the near surface soil down to a depth of 1 m.

## Diurnal periodic heating of a half-space

The surface temperature of the earth regularly changes over time due to the day-night cycle and seasonal changes.

In this notebook, we use the 1D heat conduction equation

$$
\begin{equation}
\frac{\partial T}{\partial t} = \alpha \frac{\partial^2 T}{\partial z^2} \notag
\end{equation}
$$

with the thermal diffusivity $\alpha$ and temperature $T$, to compute the influence of diurnal periodic heating of the surface 

$$
\begin{equation}
T_{surf} = T_{avg} + \Delta T cos(\omega t) \tag{1}
\end{equation}
$$

on the temperature distribution in the near surface soil. Here, $T_{avg}$ denotes the average temperature over the day, $\Delta T$ the amplitude of the periodic temperature variation and the circular frequency $\omega$ related to the period $\tau$:

$$
\begin{equation}
\omega = \frac{2 \pi}{\tau} \notag
\end{equation}
$$

Let's take a deeper look into the problem.

### Setting up the problem

Lets assume that we have an average surface temperature $T_{avg} = 10 ^oC$, temperature variations of $\Delta T = 5^oC$ and a diurnal period $\tau = 24\; h$. The half-space extends down to a depth of $L = 2\; m$ with an initial temperature of $T_{init} = 3\; ^oC$. 

Additionally, the [thermal diffusivity](http://en.wikipedia.org/wiki/Thermal_diffusivity) of the mantle rock is $\alpha=1\times10^{-6} {\rm m}^2/{\rm s}$. As usual, start by importing some libraries and setting up the discretization.

In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline

In [ ]:
# Set the font family and size to use for Matplotlib figures.
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

We'll begin by defining a few parameters ...

- depth of the half-space $L = 1\; \text{m}$
- a spatial grid with $nz = 100\; \text{points}$ in z-direction
- thermal diffusivity $\alpha = 1\times10^{-6}\; m^2/s$
- initial temperature of the half-space $T_{init} = 3^oC$
- average surface temperature $T_{avg} = 10 ^oC$
- temperature variations of $\Delta T = 5^oC$
- diurnal period $\tau = 24\; h$

In [ ]:
# Set parameters
L =         # DEFINE DEPTH OF HALF-SPACE HERE [m]!
nz =        # DEFINE NUMBER OF GRIDPOINTS NZ HERE! 
dz = L / (nz - 1)  # spatial distance between two consecutive locations [m]
alpha =     #  DEFINE THERMAL DIFFUSIVITY OF ROCK HERE [m^2/s]!

# Define the depth locations [m]
z = numpy.linspace(0.0, L, num=nz)

# Set the initial temperature in the half-space
T_init =     # INITIAL TEMPERATURE OF THE HALF-SPACE [°C]
T0 = numpy.ones(nz)
T0 = T0 * T_init

# DEFINE PARAMETERS FOR PERIODIC DIURNAL BOUNDARY CONDITION AT THE SURFACE HERE!
T_avg =       # average temperature [°C]
DT =          # amplitude of periodic temperature variations [°C]
tau =         # period of periodic temperature variations [s]

To solve the 1D heat conduction equation, we are using the forward-time, centered-space **FTCS** discretization. You should work it out on a piece of paper yourself (if you can't do it without looking it up, it means you need to do this more!).

$$
\begin{equation}
\frac{T_{i}^{n+1}-T_{i}^{n}}{\Delta t}=\alpha\frac{T_{i+1}^{n}-2T_{i}^{n}+T_{i-1}^{n}}{\Delta z^2} \notag
\end{equation}
$$

To obtain the temperature at the next time step, $T^{n+1}_i$, from the known information at the current time step, we compute

$$
\begin{equation}
T_{i}^{n+1}=T_{i}^{n}+\frac{\alpha\Delta t}{\Delta z^2}(T_{i+1}^{n}-2T_{i}^{n}+T_{i-1}^{n}) \notag
\end{equation}
$$

In the [first lesson of module 5](https://nbviewer.jupyter.org/github/numerical-mooc/numerical-mooc/blob/master/lessons/02_spacetime/02_03_1DDiffusion.ipynb), we discretized the diffusion equation  with a forward-time, centered-space scheme, subject to the following stability constraint:

$$
\begin{equation}
\alpha \frac{\Delta t}{(\Delta z)^2} \leq \frac{1}{2} \notag
\end{equation}
$$

##### Exercise 1

Modify the following **FCTS** FD code to implement the periodic boundary condition based on eq. (1) at the surface for given parameters $T_{avg},\;\Delta T,\;\tau$:

In [ ]:
def ftcs_cool_diurnal(T0, nt, nz, dt, dz, alpha, T_avg, DT, tau):
    """
    Computes and returns the depth-temperature distrubtion (geotherm)    
    for the half-space cooling problem, according to a provided number 
    of time steps, a given initial temperature and thermal diffusivity.
    The diffusion equation is integrated using forward differencing in 
    time and central differencing in space.
    
    Parameters
    ----------
    T0 : numpy.ndarray
        The initial temperature as a 1D array of floats.
    nt : integer
        The number of time steps to compute.
    nz : integer
        The number of spatial grid points.    
    dt : float
        The time-step size to integrate.
    dz : float
        The distance between two consecutive locations.
    alpha : float
        The thermal diffusivity of the rock.
    
    Returns
    -------
    Tstore : numpy.ndarray
        The temperature evolution [°C] as a 2D array of floats.
    time : numpy.ndarray
        time [s] as a 1D array of floats.    
    """
    
    T = T0.copy()
    sigma = alpha * dt / dz**2
    
    # Initialize array to store T-model at each time step n ...
    Tstore = numpy.zeros([nz,nt])
    
    # ... and time 
    time = numpy.zeros(nt)
    
    # DEFINE CIRCULAR FREQUENCY OMEGA HERE!
    omega = 
    
    for n in range(nt): # loop over time steps
        
        Tn = T.copy()
        for i in range(1,nz-1): # loop over spatial grid
            
            T[i] = (Tn[i] + sigma * (Tn[i+1] - 2.0 * Tn[i] + Tn[i-1]))
            
        # APPLY DIURNAL PERIODIC BOUNDARY CONDITION EQ. (1) HERE!
        T[0] =
        
        # Store temperature model at time step n in 2D array Tstore ...
        Tstore[:,n] = T
        
        # ... and time 
        time[n] = n * dt
        
    return Tstore, time

We are all set to run! Lets compute the temperature evolution during one diurnal cycle of 24 hours. First, you have to define the maximum cool time **Tmax = 24 h**. To get correct results, do not forget to convert the units of Tmax from [h] to [s].

Next, we define a time step `dt` that satisfies the stability constraint and calculate the number of time steps `nt` for the given `Tmax` and `dt`. Compute the temperature evolution during 24 hours using the FD code `ftcs_cool_diurnal` 

In [ ]:
# DEFINE MAXIMUM COOLING TIME in [h]
Tmax =  

# CONVERT Tmax UNITS [h] -> [s] HERE!
Tmax = 

# Set the time-step size based on CFL limit.
sigma = 0.5
dt = sigma * dz**2 / alpha  # time-step size
nt = (int)(Tmax/dt)         # number of time steps

# Compute the temperature profile in the lithosphere and asthenosphere
Tstore, time = ftcs_cool_diurnal(T0, nt, nz, dt, dz, alpha, T_avg, DT, tau)

Next, we plot the temperature evolution as image plot ...

In [ ]:
# Plot the temperature evolution of the half-space after Tmax = 24 hours
pyplot.figure(figsize=(15.0, 5.))

pyplot.imshow(numpy.flipud(Tstore),extent=[0,Tmax/(60*60),0,L],aspect=10,cmap='magma')
cb = pyplot.colorbar()
pyplot.ylabel('Depth [m]')
pyplot.xlabel('Time [h]')
cb.set_label(label='Temperature [°C]',weight='bold')
pyplot.gca().invert_yaxis()

... and as line plots at $\text{depth} = 1\; \text{cm}, 2\; \text{cm}, 5\; \text{cm}, 10\; \text{cm}, 20\; \text{cm}, 40\; \text{cm}, 80\; \text{cm}$, respectively:

In [ ]:
# Plot the temperature evolution of the half-space in 1 cm, 2 cm, 5 cm , 
# 10 cm, 20 cm, 40 cm and 80 cm, respectively.
pyplot.figure(figsize=(10.0, 6.0))
pyplot.xlabel('Time [h]')
pyplot.ylabel('Temperature [°C]')
pyplot.grid()

# Temperature evolution in 1 cm depth
n1cm = (int)(0.01/dz)
pyplot.plot(time/(60.*60.),Tstore[n1cm,:], color='C0', linestyle='-', linewidth=3, label='1 cm depth')

# Temperature evolution in 2 cm depth
n2cm = (int)(0.02/dz)
pyplot.plot(time/(60.*60.),Tstore[n2cm,:], color='C1', linestyle='-', linewidth=3, label='2 cm depth')

# Temperature evolution in 5 cm depth
n5cm = (int)(0.05/dz)
pyplot.plot(time/(60.*60.),Tstore[n5cm,:], color='C2', linestyle='-', linewidth=3, label='5 cm depth')

# Temperature evolution in 10 cm depth
n10cm = (int)(0.1/dz)
pyplot.plot(time/(60.*60.),Tstore[n10cm,:], color='C3', linestyle='-', linewidth=3, label='10 cm depth')

# Temperature evolution in 20 cm depth
n20cm = (int)(0.2/dz)
pyplot.plot(time/(60.*60.),Tstore[n20cm,:], color='C4', linestyle='-', linewidth=3, label='20 cm depth')

# Temperature evolution in 40 cm depth
n40cm = (int)(0.4/dz)
pyplot.plot(time/(60.*60.),Tstore[n40cm,:], color='C5', linestyle='-', linewidth=3, label='40 cm depth')

# Temperature evolution in 80 cm depth
n80cm = (int)(0.8/dz)
pyplot.plot(time/(60.*60.),Tstore[n80cm,:], color='C6', linestyle='-', linewidth=3, label='80 cm depth')

pyplot.legend()
pyplot.xlim(0,Tmax/(60.*60.))

Describe and discuss the results.

## What we learned:

- How to implement a periodic diurnal boundary condition at the surface
- Time evolution of the temperature field in the upper 1 m of the soil